## Introduction:

This Python script utilizes Google Gemini, a multimodal LLM (Large Language Model), to analyze a video and identify scene changes, specifically focusing on detecting potential points for ad insertion. The script prompts Gemini to select the 10 most suitable scene transitions across the video, distributing them throughout the beginning, middle, and end. These points are chosen to minimize interruptions for viewers and ensure optimal ad placement.

The Gemini model provides metadata for each identified scene change, including the timestamp, reason for the scene change, summary, transition type, narrative role, dialogue intensity, and character types. This metadata is returned in a consistent JSON structure, thanks to Gemini's controlled output feature, making it suitable for downstream processing without additional transformations. The script converts this structured JSON response into a Pandas DataFrame for storage in a database like BigQuery for future analysis.

In [1]:
import json
import pandas as pd

import vertexai  # Import the Vertex AI library for initializing and using the Gemini model
from google.cloud import storage  # Google Cloud Storage client for handling GCS
from vertexai.generative_models import (
    GenerationConfig,        # Configuration settings for Gemini's response generation
    GenerativeModel,         # Class representing the generative model used for generating responses
    Part,                    # Used to define parts of multimodal content like videos
    Content,                 # Represents the content used in the generation request
    GenerationResponse,      # Structure to handle the response from Gemini
)


In [3]:
# Set the GCP project and location where Vertex AI is being used
project_id = "lab-project-426319"#"[project-id]"
location = "us-central1"

# Initialize Vertex AI with the project ID and location to use the Gemini model
vertexai.init(project=project_id, location=location)



In [4]:
# Define the prompt that will be sent to Gemini.
# This prompt explains the task to analyze the video and identify the best scene changes for ad placement.
prompt = '''
       I have a video that I need you to analyze for ad placement by detecting scene changes, 
       also known as shot boundaries. I need to identify the 10 best scene changes across the 
       entire movie, which are the best potential points for ad placement as they minimize 
       interruptions for viewers. These scene changes should be selected from all parts of the movie: 
       the beginning, middle, and the very end. Make sure you distribute the selected scenes evenly across 
       the entire movie.
       For each of these scene changes, please provide:

        timestamp: The exact timestamps indicating where the scene change occurs. Make sure that the timestamp of scenes are matched those in the original movie,
        reflecting its position accurately. The timestamps must exactly match those in the original movie.
        
        reason: The reason why this is a scene change and why it is a good location for ad placement. the reason 
        should be very specific. Summarize the story after and before the scene and explain why 
        between these two scenes is a good place for an ad.
        
        summary: A brief summary of the scene before the change.
        
        transition_feeling: The main feeling that the transition makes in viewers like excitement, peace, fear, etc.
        
        transition_type: The method used to switch from one scene to another like cuts, fades, dissolves, etc.
        
        narrative_type: The main role or significance of the scene in the storyline like pivotal, climatic, conflict, etc.
        
        dialogue_intensity: The amount and intensity of dialogue in the scene like monologue, dialogue, narration, debate, etc.

        characters_type: The types of the most important character involved in the scene transition like protagonist, antagonist, supporting, etc.
        
        scene_categories:  Classification of the scene before the change into the categories such as action, drama, comedy, etc.
      '''      

# Define the expected response schema to ensure the output JSON is structured correctly
response_schema = {
    "type": "array",
    "items": {
        "type": "object",
        "properties": {
            "timestamp": {"type": "string"},
            "reason": {"type": "string"},
            "transition_feeling": {"type": "string"},
            "transition_type": {"type": "string"},
            "narrative_type": {"type": "string"},
            "dialogue_intensity": {"type": "string"},
            "characters_type": {"type": "string"},
            "scene_categories": {"type": "string"},
        },
        # Ensure that these properties are always present in the output
        "required": ["timestamp", "reason","transition_feeling","transition_type","narrative_type",
                    "characters_type","scene_categories"],
    },
}


In [5]:
# Specify the version of the Gemini model to be used
model_id = "gemini-1.5-pro-001"  
model = GenerativeModel(model_id)

# Set up the generation configuration to control Gemini's response
generation_config = GenerationConfig(
    temperature=0,  # Set the temperature to 0 for consistent output
    response_mime_type="application/json",  # Expect the response to be in JSON format
    response_schema=response_schema  # Use the predefined response schema for structured output
)

In [6]:
# Define the URL of the video file stored in Google Cloud Storage
video_file_url = 'gs://video_demo_test/wakeup_princess.mp4'

# Load the video file from Google Cloud Storage as an input to Gemini
video_file = Part.from_uri(video_file_url, mime_type="video/mp4")

# Combine the video file and the prompt into a single input to the Gemini model
contents = [video_file, prompt]

# Generate content from Gemini by passing the contents and the generation configuration
response = model.generate_content(contents, generation_config=generation_config)


In [7]:
# Parse the JSON response from Gemini into a Python dictionary
json_response = json.loads(response.text)

# Convert the JSON response into a Pandas DataFrame for easier analysis and storage
df_response = pd.DataFrame(json_response)